# Estimating Gender Discrimination in the Workplace

In this exercise we'll use data from the 2018 US Current Population Survey (CPS) to try and estimate the effect of being a woman on workplace compensation. 

Note that our focus will be *only* on differential compensation in the work place, and as a result it is important to bear in mind that our estimates are not estimates of *all* forms of gender discrimination. For example, these analyses will not account for things like gender discrimination in terms of *getting* jobs. We'll discuss this in more detail below.

In [2]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")
pd.set_option("mode.copy_on_write", True)

## Exercise 1: 

Begin by downloading and importing 2018 CPS data from [https://github.com/nickeubank/MIDS_Data/tree/master/Current_Population_Survey](https://github.com/nickeubank/MIDS_Data/tree/master/Current_Population_Survey). The file is called `morg18.dta` and is a Stata dataset. Additional data on the dataset can be found by following the links in the README.txt file in the folder, but for the moment it is sufficient to know this is a national survey run in the United States.

The survey does include some survey weights we won't be using (i.e. not everyone in the sample was included with the same probability), so the numbers we estimate will not be perfect estimates of the gender wage gap in the United States, but they are pretty close.

In [3]:
data = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/blob/master/Current_Population_Survey/morg18.dta?raw=true"
)

## Exercise 2:

Because our interest is only in-the-workplace wage discrimination among full-time workers, we need to start by subsetting our data for people currently employed (and "at work", not "absent") at the time of this survey using the `lfsr94` variable, who are employed full time (meaning that their usual hours per week—`uhourse`—is 35 or above).

As noted above, this analysis will miss many forms of gender discrimination. For example, in dropping anyone who isn't working, we immediately lose any women who couldn't get jobs, or who chose to lose the workforce because the wages they were offered (which were likely lower than those offered men) were lower than they were willing / could accept. And in focusing on full time employees, we miss the fact women may not be offered full time jobs at the same rate as men. 

In [9]:
data["lfsr94"].unique()

['Disabled-Not In Labor Force', 'Retired-Not In Labor Force', 'Employed-At Work', 'Unemployed-Looking', 'Employed-Absent', 'Other-Not In Labor Force', NaN, 'Unemployed-On Layoff']
Categories (7, object): ['Employed-At Work' < 'Employed-Absent' < 'Unemployed-On Layoff' < 'Unemployed-Looking' < 'Retired-Not In Labor Force' < 'Disabled-Not In Labor Force' < 'Other-Not In Labor Force']

In [10]:
data["uhourse"].unique()

array([nan, 40., 30., -4., 36., 20., 48., 35., 44., 15., 50., 45., -1.,
       60., 17., 19., 41., 10., 70., 25., 11., 75., 18., 32.,  2., 80.,
        5., 24.,  4.,  3., 38.,  6., 56., 96., 82., 55., 52., 77., 84.,
        0., 12., 16., 42., 43., 54., 23., 53., 33., 28., 65.,  9., 37.,
       72., 26.,  7., 61., 27., 99., 58.,  8., 21., 22., 13., 47., 63.,
        1., 46., 39., 29., 14., 49., 66., 34., 57., 90., 62., 85., 31.,
       87., 68., 69., 76., 83., 98., 88., 51., 78., 64., 67., 92., 95.,
       74., 86., 81., 59., 71., 73., 91., 89., 94.])

In [18]:
subset = data[data["lfsr94"] == "Employed-At Work"]
subset = data[data["uhourse"] >= 35]

In [19]:
subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 137868 entries, 2 to 302329
Data columns (total 98 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   hhid       137868 non-null  object  
 1   intmonth   137868 non-null  category
 2   hurespli   137855 non-null  float64 
 3   hrhtype    137868 non-null  category
 4   minsamp    137868 non-null  category
 5   hrlonglk   137868 non-null  category
 6   hrsample   137868 non-null  object  
 7   hrhhid2    137868 non-null  object  
 8   serial     137868 non-null  object  
 9   hhnum      137868 non-null  int8    
 10  stfips     137868 non-null  category
 11  cbsafips   137868 non-null  int32   
 12  county     137868 non-null  int16   
 13  centcity   114693 non-null  float64 
 14  smsastat   136656 non-null  float64 
 15  icntcity   18890 non-null   float64 
 16  smsa04     137868 non-null  int8    
 17  relref95   137868 non-null  int8    
 18  age        137868 non-null  int8    
 19  spouse 

## Exercise 3

Now let's estimate the basic wage gap for the United States!

Earnings per week worked can be found in the `earnwke` variable. Using the variable `sex` (1=Male, 2=Female), estimate the gender wage gap in terms of wages per hour worked!

(You may also find it helpful, for context, to estimate the average hourly pay by dividing weekly pay by `uhourse`.)

In [21]:
subset["female"] = (subset["sex"] == 2).astype(int)
subset["average_hourly_wage"] = subset["earnwke"] / subset["uhourse"]
female_avg = subset[subset["female"] == 1].earnwke.mean()
male_avg = subset[subset["female"] == 0].earnwke.mean()
average = subset.earnwke.mean()

In [26]:
print(f'basic wage gap for the United States is ${male_avg - female_avg: .1f} with the percentage difference of {(male_avg - female_avg)/average*100:.1f}%')
print(f'to be precise average wage per week for women is ${female_avg: .1f}, in contrast to men which is ${male_avg: .1f}')

basic wage gap for the United States is $ 218.4 with the percentage difference of 19.7%
to be precise average wage per week for women is $ 987.6, in contrast to men which is $ 1206.0


## Exercise 4

Assuming 48 work weeks in a year, calculate annual earnings for men and women. Report the difference in dollars and in percentage terms.

In [27]:
subset["yearly_income"] = subset["earnwke"] * 48
female_yearly = subset[subset["female"] == 1].yearly_income.mean()
male_yearly = subset[subset["female"] == 0].yearly_income.mean()
average_yearly = subset.yearly_income.mean()

In [29]:
print(f'basic wage gap for the United States is ${male_yearly - female_yearly: .1f} with the percentage difference of {(male_yearly - female_yearly)/average_yearly*100:.1f}%')
print(f'to be precise average wage per year for women is ${female_yearly: .1f}, in contrast to men which is ${male_yearly: .1f}')

basic wage gap for the United States is $ 10482.8 with the percentage difference of 19.7%
to be precise average wage per year for women is $ 47405.6, in contrast to men which is $ 57888.4


## Exercise 5

We just compared all full-time working men to all full-time working women. For this to be an accurate *causal* estimate of the effect of being a woman in the work place, what must be true of these two groups? What is one reason that this may *not* be true?

> It might not to be true if men earn this much because they deserved that for working longer and having more experience. Maybe they have different achivements like more certifications or higher education level. If those are equal then it makes sense to conclude that there is a pay gap between men and and women.

> It is true to compare if men and women do the same work and thus their potential outcomes are the same.

## Exercise 6

One answer to the second part of Exercise 5 is that working women are likely to be younger, since a larger portion of younger women are entering the workforce as compared to older generations.

To *control* for this difference, let's now regress annual earnings on gender, age, and age-squared (the relationship between age and income is generally non-linear). What is the implied average annual wage difference between women and men? Is it different from your raw estimate? 

## Exercise 7

In running this regression and interpreting the coefficient on `female`, what is the implicit comparison you are making? In other words, when we run this regression and interpreting the coefficient on `female`, we're basically pretending we are comparing two groups and assuming they are counter-factuals for one another. What are these two groups?

## Exercise 8

Now let's add to our regression an indicator variable for whether the respondent has at least graduated high school, and an indicator for whether the respondent at least has a BA. 

In answering this question, use the following table of codes for the variable `grade92`. 

Education is coded as follows:
    
![CPS Educ Codes](../images/cps_educ_codes.png)

## Exercise 9 

In running this regression and interpreting the coefficient on `female`, what is the implicit comparison you are making? In other words, when we run this regression and interpreting the coefficient on `female`, we are once more basically pretending we are comparing two groups and assuming they are counter-factuals for one another. What are these two groups?

## Exercise 10

Given how the coefficient on `female` has changed between Exercise 6 and Exercise 8, what can you infer about the educational attainment of the women in your survey data (as compared to the educational attainment of men)?

## Exercise 11

What does that tell you about the *potential outcomes* of men and women before you added education as a control?

## Exercise 12

Finally, let's include *fixed effects* for the type of job held by each respondent. 

Fixed effects are a method used when we have a nested data structure in which respondents belong to groups, and those groups may all be subject to different pressures. In this context, for example, we can add fixed effects for the industry of each respondent—since wages often vary across industries, controlling for industry is likely to improve our estimates. Use `ind02` to control for industry.

(Note that fixed effects are very similar in principle to hierarchical models. There are some differences [you will read about](../fixed_effects_v_hierarchical.ipynb) for our next class, but they are designed to serve the same role, just with slightly different mechanics). 

When we add fixed effects for groups like this, our interpretation of the other coefficients changes. Whereas in previous exercises we were trying to explain variation in men and women's wages *across all respondents*, we are now effectively comparing men and women's wages *within each employment sector*. Our coefficient on `female`, in other words, now tells us how much less (on average) we would expect a woman to be paid than a man *within the same industry*, not across all respondents. 

(Note that running this regression will result in lots of coefficients popping up you don't care about. We'll introduce some more efficient methods for adding fixed effects that aren't so messy in a later class -- for now, you can ignore those coefficients!)

## Exercise 13

Now that we've added industry fixed effects, what groups are we implicitly treated as counter-factuals for one another now? 

## Exercise 14

What happened to your estimate of the gender wage gap when you added industry fixed effects? What does that tell you about the industries chosen by women as opposed to men?

When you're done, please come read [this discussion](discussion_regressions_incomeineq.ipynb).